# Wstęp
Dany jest zbiór TitanicMess.tsv, który zawiera dane dotyczące pasażerów podróżujących statkiem Titanic w dniu jego katastrofy. Chcąc przystąpić do analizy takiego zbioru należy jednak najpierw sprawdzić, czy dane występują w takiej postaci jakiej oczekujemy. Może wystąpić szereg nieprawidłowości, które należy wyeliminować, aby możliwa była praca ze zbiorem. 
Pierwszym zadaniem do wykonania będzie więc znalezienie wszelkich nieprawdiłowości. Następnie nieprawidłowości te zostaną wyeliminowane.

# Analiza zbioru

In [1]:
# import bibliotek

import pandas as pd
import numpy as np
from scipy import stats
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import re

In [2]:
# wczytanie zbioru i wyświetlenie jego pierwszych rekordów

df = pd.read_csv(r"TitanicMess.tsv", sep="\t")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


### Duplikaty danych

Sprawdźmy czy dane nie zawierają duplikatów.

In [3]:
df[df.duplicated(keep=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
10,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
13,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
23,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
224,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
520,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic


Widać, że zbiór zawiera powtarzające się rekordy. Czy możemy mieć pewność, że powyżej wyświetlone zostały wszystkie duplikaty? Jednym z atrybutów zbioru jest _PassengerId_. Ma on za zadanie jednoznacznie identyfikować obiekt w zbiorze. Sprawdźmy więc, czy w zbiorze nie występują duplikaty, ale tym razem tylko według tego atrybutu.

In [4]:
df[df.duplicated(['PassengerId'], keep=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
10,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
13,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
23,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
224,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
520,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
678,225,1,1,"Hoytt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic


Tym razem znaleziono więcej wyników. Widać, że jeden z rekordów powtarza się, ale zawiera literówkę, która sprawiła, że w poprzednim przeszukaniu rekord ten nie został uzwględniony. Należy brać to pod uwagę podczas eliminacji powtarzających się danych. Poniższa funkcja pozwala sprawdzić, czy zbiorze znajdują się duplikaty według ID.

In [5]:
df['PassengerId'].is_unique

False

### Błędne typy danych

Jeśli w kolumnach, które powinny zawierać tylko wartości numeryczne pojawią się jakieś nietypowe znaki, typ danych w tej kolumnie może nie odpowiadać przewidywanemu. W tym celu należy przeszukać zbiór w poszukiwaniu kolumn o typie _object_. Następnie dla tych kolumn sprawdzić wartości. W tym celu napisano specjalne funkcje.

In [6]:
def is_number(num):
    return num.lstrip('-').replace('.','',1).replace(',','',1).replace('e-','',1).replace('e','',1).isdigit()

Funkcja __is_number__ sprawdza czy ciąg znaków podany jako argument może reprezentować liczbę.

In [7]:
def get_numbers_as_strings_columns(df, confidence, print_string_numbers=False, print_string_regulars=False, 
                                   print_counters=False, filter=None):
    
    if confidence not in range(0, 100):
        return 'Wrong confidence, specify number from 0 to 100.'
    
    columns = list()
    
    for col in df:
        if filter and col not in filter:
            continue

        if is_string_dtype(df[col]):
            counter = 0
            for cell in df[col]:
                if pd.isna(cell):
                    continue

                if type(cell) == str and is_number(cell):
                    counter += 1
                    if print_string_numbers: print(cell, ' - ', 'number')          
                else:
                    if print_string_regulars: print(cell, ' - ', 'regular') 

            if print_counters: print(col, ', Numbers counter: ', counter, ', Not null values: ', df[col].count())
            if (counter / df[col].count())*100 > confidence:
                columns.append(col)
            
    return columns

Funkcja __get_numbers_as_strings_columns__ przeszukuje obiekt _DataFrame_ w poszukiwaniu kolumn z danymi typu _object_. Następnie dla takich kolumn sprawdzane jest, kolumna zawiera wartości, które mogą być liczbami. Na podstawie parametru _confidence_ określa się, dla jakiego procentu danych w kolumnie kolumna ta uznawana jest za zbiór danych numerycznych. Np. dla parametru _confidence=95_ kolumna uznawana jest za liczbową jeśli w wyniku filtrowania, uznano przynajmniej 95% jej wartości niepustych jako liczbowe. Parametr taki jest potrzebny, ponieważ dane mogą zawierać błędy takie jak przypadkowo wpisane litery. 

Teraz sprawdzimy, które kolumny mogą zawierać takie dane.

In [8]:
get_numbers_as_strings_columns(df, 95, print_counters=True)

Name , Numbers counter:  0 , Not null values:  892
Sex , Numbers counter:  0 , Not null values:  892
Age , Numbers counter:  719 , Not null values:  719
Ticket , Numbers counter:  661 , Not null values:  892
Fare , Numbers counter:  891 , Not null values:  892
Cabin , Numbers counter:  0 , Not null values:  207
Embarked , Numbers counter:  0 , Not null values:  890
ship , Numbers counter:  0 , Not null values:  892


['Age', 'Fare']

Widać, że algorytm wytypował kolumny _Age_ oraz _Fare_ jako zbiory liczb. Dla kolumny _Age_ wszystkie wartości uznane zostały za liczbowe, natomiast dla kolmuny _Fare_ wszystkie oprócz jednej. Możemy jeszcze raz użyć powyższej funkcji do wypisania danych, które uznane zostały za nieliczbowe.

In [9]:
get_numbers_as_strings_columns(df, 95, print_string_regulars=True, print_counters=False, filter=('Fare'))

15,9a  -  regular


['Fare']

Widać, że w kolumnie znajdowała się wartość liczbowa z literą. Trzeba będzie więc oczyścić wytypowane kolumny z nieporządanych znaków.  

### Błędy w wartościach dla danych jakościowych

Sprawdźmy po ile unikalnych wartości zawiera każda z kolumn zbioru.

In [10]:
print(df.nunique())

PassengerId    888
Survived         2
Pclass           3
Name           889
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           250
Cabin          145
Embarked         6
ship             1
dtype: int64


Sprawdźmy teraz jakie są to wartości, dla kolumn gdzie liczba unikalnych wartości jest niewielka. Możemy przefiltrować wyniki w taki sposób, otrzymać tylko kolumny z obiektami typu Object. W ten sposób wyselekcjonujemy kolumny, do sprawdzenia.

In [11]:
columns = [col for col in df if (df[col].nunique() < 10) and (is_string_dtype(df[col]))]
print(columns)

['Sex', 'Embarked', 'ship']


W zbiorze się kilka takich kolumn. Wcześniej pokazano, że np. dla kolumny _Sex_ zbiór posiada 6 unikalnych wartości. To jasno pokazuje, że w zbiorze mogą być błędy. Teraz Sprawdźmy, jakie są to wartości. Celowo pomijamy wartości puste.

In [12]:
for col in columns:
    print(col, ': ', df[col].dropna().unique())

Sex :  ['male' 'female' 'malef' 'mal' 'fem' 'femmale']
Embarked :  ['S' 'C' 'Q' 'So' 'Co' 'Qe']
ship :  ['Titanic']


Widać, że niektóre z wartości zawierają po prostu literówki - są błędne. Takie wartości możemy po prostu usunąć. 

### Nieporządane znaki w kolumnie _Name_

Jedna z kolumn zbioru zawiera imiona pasażerów. Można je wstępnie przefiltrować w poszukiwaniu nietypowych znaków.

In [13]:
for name in df['Name']:
     if re.search(r'[^A-Za-z\s,.\"\'\-()/]', name):
        print('imię:', name)
        print(re.search(r'[^A-Za-z\s,.\"\'\-()/]', name))

imię: Sandstrom, Miss. Marguerite Ru&5$$
<re.Match object; span=(30, 31), match='&'>
imię: Sandstrom, Miss. Marguerite Ru&5$$
<re.Match object; span=(30, 31), match='&'>
imię: Sandstrom, Miss. Marguerite Ru&5$$
<re.Match object; span=(30, 31), match='&'>


Znalezio jedno wystąpienie zawierające znaki nieporządane (powtórzone 3 razy, ponieważ usuwanie duplikatów następuje później). Należy je w takim razie usunąć.

Istnieje tutaj możliwość przefiltrowania według wielu kategorii. Można by na przykład usunąć pseudonimy czy rozdzielić kolumnę na dwie, jednak to wykracza poza niezbędne czynności mające na celu wyczyszczenie zbioru.

### Kolumny zbędne

Teraz sprawdzone zostanie występowanie kolumn zbędnych w zbiorze. Szukamy kolumn o tylko jednej wartości.

In [14]:
for col in df:
    if df[col].nunique() == 1:
        print(col)

ship


Widać, że ostatnia kolumna zbioru zawiera niepotrzebną i powtarzającą się informację.

In [15]:
df['ship'].describe()

count         892
unique          1
top       Titanic
freq          892
Name: ship, dtype: object

Każdy obiekt posiada tą samą wartość dla tego atrybutu, będącego informacją dotyczącą statku jakim podróżowali pasażerowie. Wiedząc na wstępie, że dane dotyczą pasażerów jednego statku i widząc, że są one powtarzalne oraz zbędne, można je w całości usunąć ze zbioru.

### Atrybuty numeryczne, które powinny być nominalne

Można sprawdzić czy zbiór zawiera atrybuty o wartościach 1 lub 0.

In [16]:
df.columns[df.isin([0,1]).all()]

Index(['Survived'], dtype='object')

Atrybut _Survived_ zawiera wartości 0 i 1.

In [17]:
df['Survived'].value_counts()

0    547
1    345
Name: Survived, dtype: int64

Możemy zamienić jego wartości na nominalne. Widać również, że atrybut _Pclass_ nie powinien być traktowany jako numeryczny.

In [18]:
df['Pclass'].value_counts()

3    491
1    217
2    184
Name: Pclass, dtype: int64

### Wartości odstające

Sprawdźmy czy w zbiorze występują jakieś wartości odstające dla zmiennych numerycznych. Najłatwiej to zrobić biorąc pod lupę atrybut, którego zakres wartości mniej więcej znamy. W tym zbiorze takim atrybutem jest _Age_. 

In [19]:
for val in df.Age.dropna():
    try:
        if not (0 <= int(val) <= 100):
            print(val)
    except Exception:
        continue

4435
-3
-12
250


Sprawdzony atrybut zawiera wartości odstające, dlatego więcej atrybutów na razie nie sprawdzamy. W dalszej części zastosujemy algorytm, który przeszuka wszystkie kolumny w poszukiwaniu takich wartości.

### Brakujące wartości
Sprawdźmy teraz, czy w zbiorze występują brakujące wartości.

In [20]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
ship             0
dtype: int64

Wygląda na to, że zbiór zawiera brakujące wartości dla atrybutów Age, Cabin oraz Embarked.



# Czyszczenie danych

Teraz wykonane zostaną wszystkie operacje mające na celu przygotowanie zbioru do wykonywania analiz. **Zadbano o to, aby poniższy kod w miarę możliwości dało się wykorzystać dla innych zbiorów. Niektóre problemy wymagały jednak indywidualnego podejścia. Umieszczono także krótkie komentarze dotyczące wykonywanych operacji dla tego konkretnego zbioru, którego zawartość omawiana była w powyższej analizie.**  

### Usuwanie duplikatów
Usunięte zostaną duplikaty danych.

In [21]:
df = df.drop_duplicates(ignore_index=True)
df['PassengerId'].is_unique

False

Powyższa funkcja zwraca false, ponieważ nie usunięto duplikatów według numeru ID. Ta operacja zostanie wykonana poniżej.

In [22]:
df = df.drop_duplicates('PassengerId', ignore_index=True)
df['PassengerId'].is_unique

True

Teraz w zbiorze duplikaty już nie występują.

### Czyszczenie i zmiana typów danych dla kolumn z wartościami numerycznymi zapisanymi jako string

W celu oczyszcenia danych napisano funkcję. Dla kolumn, których nazwy przekazywane są jako parametr następuje czyszczenie danych. Na koniec zmieniamy typ danych dla kolumny.

In [23]:
def clean_numeric_columns(df, columns):
    for col in columns:
        for index_row, cell in enumerate(df[col]):
            if pd.isna(cell): continue
            
            # zamiana przecinków na kropki
            cell = cell.strip().replace(',', '.')
            
            # usuwanie nieporządanych znaków (zostawia tylko liczby, kropkę dla ułamków, minusy) 
            cell = re.sub(r'[^-\d\.]', '', cell)

            # modyfikacja (podmiana elementu)
            try:
                df.loc[index_row, col] = cell
            except Exception as ex:
                print(ex)
            
        # konwersja na typ liczbowy
        try:
            df[col] = pd.to_numeric(df[col])
        except Exception as ex:
            print("Something went wrong parsing ", col, ', ex: ', ex)

Odpowiednie kolumny do przekazania dla funkcji otrzymamy wykorzystując funkcję __get_numbers_as_strings_columns__ napisaną wcześniej.

In [24]:
cols = get_numbers_as_strings_columns(df, 95)
clean_numeric_columns(df, cols)
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
ship            object
dtype: object

Typ kolumn wskazuje na to, że dane są już w poprawnym formacie.

### Usuwanie błędnych wartości dla danych jakościowych

Aby usunąć niechciane dane wykorzystana zostanie funkcja stworzona specjalnie do rozwiązania tego typu problemu.

In [25]:
def remove_categorical_mistakes(df, factor, min_perc_occurences, filter_func=lambda _: True, print_info=True):                 
    # znajdź kolumny o [x < factor] unikalnych wartościach
    columns = [col for col in df if (df[col].nunique() < factor) and (filter_func(df[col]))]

    for col in columns:
        for val in df[col].dropna().unique():
            counter = df[col].value_counts()[val]
            if print_info: print(col, ', value: ',val, ' =  ', counter, 'times')
            
            if not counter/df[col].count()*100 > min_perc_occurences:
                df[col] = df[col].replace(val, np.nan)
                if print_info: print('Deleting value:', val)

Powyższa funkcja zaczyna od wyszukania odpowiednich kolumn. Przyjmuje jako parametr _factor_ liczbę określającą maksymalną liczbę unikalnych wartości dla wyszukiwanych kolumn. Można też jako parametr przekazać funkcje, która dodatkowo filtruje kolumny. Następnie liczone są wystąpienia każdej z wartości. Jeśli wystąpienia te będą stanowić procentowo mniejszą część zbioru niż procent podany jako parametr *min_perc_occurences*, to wartości takie zostaną usunięte.

Wykorzystamy teraz powyższą funkcję. Przekazane parametry pozwolą na wyszukanie kolumn z danymi typu _object_ posiadających mniej niż 10 unikalnych wartości. Wszystkie wartości stanowiące mniej niż podany 1% całej kolumny zostaną usunięte.

In [26]:
print(df.nunique(), '\n')
remove_categorical_mistakes(df, 10, 1, filter_func=is_string_dtype)
df.nunique()

PassengerId    888
Survived         2
Pclass           3
Name           888
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           249
Cabin          145
Embarked         6
ship             1
dtype: int64 

Sex , value:  male  =   574 times
Sex , value:  female  =   310 times
Sex , value:  malef  =   1 times
Deleting value: malef
Sex , value:  mal  =   1 times
Deleting value: mal
Sex , value:  fem  =   1 times
Deleting value: fem
Sex , value:  femmale  =   1 times
Deleting value: femmale
Embarked , value:  S  =   639 times
Embarked , value:  C  =   167 times
Embarked , value:  Q  =   76 times
Embarked , value:  So  =   2 times
Deleting value: So
Embarked , value:  Co  =   1 times
Deleting value: Co
Embarked , value:  Qe  =   1 times
Deleting value: Qe
ship , value:  Titanic  =   888 times


PassengerId    888
Survived         2
Pclass           3
Name           888
Sex              2
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           249
Cabin          145
Embarked         3
ship             1
dtype: int64

Jak widać powyżej kolumny zostały wyczyszcone z błędnych danych.

### Usuwanie nieporządanych znaków z imion pasażerów

Znaki nieporządane znalezione wcześniej zostaną usunięte przy pomocy wyrażeń regularnych. 

In [27]:
for index, name in enumerate(df['Name']):
     if re.search(r'[^A-Za-z\s,.\"\'\-()/]', name):
        print('Oryginał:', name)
        print(re.search(r'[^A-Za-z\s,.\"\'\-()/]', name))
        striped_name = re.sub(r'[^A-Za-z\s,.\"\'\-()/]', '', name)
        print('Poprawione:',striped_name)
        df.at[index, 'Name'] = striped_name

Oryginał: Sandstrom, Miss. Marguerite Ru&5$$
<re.Match object; span=(30, 31), match='&'>
Poprawione: Sandstrom, Miss. Marguerite Ru


### Usuwanie kolumn zbędnych

In [28]:
for col in df:
    if df[col].nunique() == 1:
        df.drop(columns=col, inplace=True)

Sprawdźmy czy kolumna została usunięta. Jak widać poniżej kolumna zostaje usunięta.

In [29]:
df.nunique()

PassengerId    888
Survived         2
Pclass           3
Name           888
Sex              2
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           249
Cabin          145
Embarked         3
dtype: int64

### Zamiana wartości numerycznych na nominalne

Przeszukane zostaną wszystkie kolumny w poszukiwaniu zbiorów o wartościach binarnych (0 oraz 1).

In [30]:
for col in df.columns[df.isin([0,1]).all()]:
    df[col] = df[col].replace({0:'False', 1:'True'})
    
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Wartości 0 oraz 1 zostały zmienione na False oraz True w tym przypadku tylko dla kolumny _Survived_. Teraz zamienimy atrybut Pclass na nieliczbowy.

In [31]:
def class_to_str(num):
    if num == 1:
        return '1st'
    elif num == 2:
        return '2nd'
    elif num == 3:
        return '3rd'
    else:
        return str(num) + 'th'

df['Pclass'] = df['Pclass'].astype(object)
df['Pclass'] = df['Pclass'].apply(class_to_str)
print(df['Pclass'].unique())

['3rd' '1st' '2nd']


### Wartości odstające

Aby znaleźć wartości odstające wykorzystamy poniższą funkcję. Wykorzystuje ona kwantyle do odnalezienia wartości odstających. Przeszukanie następuje dla wszystkich kolumn numerycznych z wyjątkiem kolumny z ID.

In [32]:
def delete_outliers(df, min_quant, max_quant, include_id=False, print_info=True):
    indexes = set()
    numeric_columns = [col for col in df if is_numeric_dtype(df[col])]
    if print_info: print('Numeric columns: ', numeric_columns, '\n')
    
    for col in df[numeric_columns]:
        if not include_id and 'id' in col.lower(): continue
        print(col, '\n')

        min_thresold = df[col].quantile(min_quant)
        if print_info: print('Min thresold:', min_thresold)

        max_thresold = df[col].quantile(max_quant)
        if print_info: print('Max thresold:', max_thresold)

        for index, cell in enumerate(df[col]):
            if pd.isnull(cell): continue
            if not (min_thresold <= cell <= max_thresold):
                if print_info: print('\t Deleting row for', col, 'value:', cell)
                indexes.add(index)

        if print_info: print('\n', 25*'-', sep='')

    df.drop(list(indexes), inplace=True)

In [33]:
delete_outliers(df, 0.002, 0.998)

Numeric columns:  ['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare'] 

Age 

Min thresold: -1.5362400000000003
Max thresold: 177.24000000000046
	 Deleting row for Age value: 4435.0
	 Deleting row for Age value: -3.0
	 Deleting row for Age value: -12.0
	 Deleting row for Age value: 250.0

-------------------------
SibSp 

Min thresold: 0.0
Max thresold: 8.0

-------------------------
Parch 

Min thresold: 0.0
Max thresold: 5.0

-------------------------
Fare 

Min thresold: 0.0
Max thresold: 512.3292
	 Deleting row for Fare value: -90.0

-------------------------


Powyżej widzimy, dla jakich wartości poszczególnych zmiennych unsunięto całe rekordy w zbiorze.

### Brakujące wartości

Teraz zajmiemy się brakującymi wartościami. Poniższa funkcja dla atrybutów numerycznych zastępuje brakujące wartości średnią. Dla innych atrybutów, jeśli liczba brakujących wartości przekracza podany parametrem procent wszystkich wierszy, to cała kolumna jest usuwana. W przeciwnym wypadku usuwane są tylko wiersze gdzie występują brakujące wartości.

In [34]:
def handle_missing_values(perc_for_drop, print_info=True):
    
    columns_to_drop = list()
    columns_to_clear = list()
    
    for col in df:
        if df[col].isnull().sum() > 0:
            if is_numeric_dtype(df[col]):
                df[col].fillna(df[col].mean(), inplace=True)
                print('Replacing values with mean for column:', col)
            else:
                ratio = df[col].isnull().sum() / len(df)
                if ratio*100 > perc_for_drop:
                    columns_to_drop.append(col)
                else:
                    columns_to_clear.append(col)

    print('Rows number before removal:', len(df))
    df.drop(columns=columns_to_drop, inplace=True)
    df.dropna(subset=columns_to_clear, inplace=True)
    print('Rows number after removal:', len(df))
    print('Deleted columns:', columns_to_drop)
    
print(df.isnull().sum(), '\n')
handle_missing_values(50)
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              3
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          681
Embarked         6
dtype: int64 

Replacing values with mean for column: Age
Rows number before removal: 883
Rows number after removal: 874
Deleted columns: ['Cabin']
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


Powyżej widać, że wartości brakujące zostały usunięte.

# Podsumowanie

Udało się przeprowadzić następujące operacje w celu oczyszczenia zbioru danych:
1. Usuwanie duplikatów
2. Czyszczenie i zmiana typów danych dla kolumn z wartościami numerycznymi.
3. Usuwanie błędnych wartości dla danych jakościowych.
4. Usuwanie nieporządanych znaków z imion pasażerów
5. Usuwanie kolumn zbędnych.
6. Zamiana wartości numerycznych na nominalne
7. Usuwanie wartości odstające
8. Usuwanie brakujących wartości

Oczyszczony zbiór zapisujemy w formacie csv.

In [35]:
df.to_csv('TitanicCleaned.tsv', sep='\t')